# Setup Perfmon Data Collection

This notebook contains instructions on how to setup perfmon data collection on each server to be monitored, and import the data into Central Monitoring Server.

We create 2 Grafana dashboards on top of collected data using Central Server.

## Instructions -

1. Part 01 - Setup Baselining of SqlServer using Perfmon & Powershell
2. Part 02 - Configure Grafana for Visualization on baselined data

  

Initiatize **$Servers** with list of Server Names where Perfmon Baselining has to be done.

In [6]:
$Servers = @('localhost','196.1.115.196')

# Part 01 - Setup Baselining of SqlServer using Perfmon & PowerShell

## 1\. Create \[DBA\] database using below script. Make sure to change the data & log files path.

If a similar DBA database exists where data collection should be performed, then this step can be skipped.

In [ ]:
$sqlCreateDatabase = @"
create database [DBA] 
--on (name = 'DBA', filename = 'E:\MSSQL14.V17\Data\DBA.mdf' , SIZE = 500MB , FILEGROWTH = 200MB )
--log on (name = 'DBA_log', filename = 'E:\MSSQL14.V17\Log\DBA_log.ldf' , SIZE = 500MB , FILEGROWTH = 200MB )
go
"@

Invoke-DbaQuery -SqlInstance $Servers -Query $sqlCreateDatabase;

## 2\. Start Perfmon data collection using template **NonSql-Files\\DBA\_PerfMon\_NonSQL\_Collector\_Template.xml** inside Perfmon folder.

This directory should be at least 4 gb of size.

In [7]:
$repoFolder = "$(Get-Location)\SqlServer-Baselining-Grafana"
$data_collector_template_path = "$repoFolder\NonSql-Files\DBA_PerfMon_NonSQL_Collector_Template.xml";

$collector_root_directory = "$repoFolder\Perfmon";
if(Test-Path $repoFolder) {"'$collector_root_directory' path already exists"} else {New-Item $collector_root_directory -ItemType Directory}

$data_collector_set_name = 'DBA';
$log_file_path = "$collector_root_directory"
$file_rotation_time = '04:00:00'
$sample_interval = '00:00:05'
$version_format = 'yyyyMMdd\-HHmmss'

foreach($srv in $Servers)
{
    logman import -name "$data_collector_set_name-[$srv]" -xml "$data_collector_template_path"
    logman update -name "$data_collector_set_name-[$srv]" -f bin -cnf "$file_rotation_time" -o "$log_file_path\$($srv)__" -si "$sample_interval"
    logman start -name "$data_collector_set_name-[$srv]"
}
<#
logman stop -name "$data_collector_set_name"
logman delete -name "$data_collector_set_name"
#>


'D:\GitHub-Personal\SqlServer-Baselining-Grafana\Perfmon' path already exists


The command completed successfully.


The command completed successfully.


The command completed successfully.


The command completed successfully.


The command completed successfully.


The command completed successfully.
